# Balanced Risk Set Matching
Zeus D. Elderfield

Link to article: [Balanced Risk Set Matching: Journal of the American Statistical Association](https://doi.org/10.1198/016214501753208573)

## Instructions:
- Read the journal provided.
- Develop a Python implementation of the procedures in the journal.
- Deadline is before premidterm week.

### Import necessary libraries

In [354]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.spatial.distance import cdist # Computes distance between points of two collections
from pyscipopt import Model # Imported for integer programming problem

### Set global variables & seed

In [355]:
np.random.seed(24)

number_of_patients = 200
evaluation_years = 4
evaluation_months = evaluation_years * 12

### Load initial evaluation data upon patient entry

In [356]:
baseline = pd.DataFrame({
    "patient_id": np.arange(0, number_of_patients),
    "gender": np.random.choice(['M', 'F'], number_of_patients),
    "pain": np.random.randint(0, 10, number_of_patients),
    "urgency": np.random.randint(0, 10, number_of_patients),
    "frequency": np.random.randint(0, 20, number_of_patients)
})

baseline.head()

,patient_id,gender,pain,urgency,frequency
0,0,M,1,7,16
1,1,F,1,8,19
2,2,M,8,8,14
3,3,F,3,2,18
4,4,F,5,4,17


### Load evaluation data every 3 months

In [357]:
evaluations = pd.DataFrame()

for patient_id in range(number_of_patients):
    chosen_treatment_time = np.random.choice(list(np.arange(3, evaluation_months + 1, 3)) + [None])
    
    # Evaluate every 3 months up to 4 years
    for month in range(3, evaluation_months + 1, 3):
        pain = np.random.randint(0, 10, 1)
        urgency = np.random.randint(0, 10, 1)
        frequency = np.random.randint(0, 20, 1)
        time_since_entry = month

        if chosen_treatment_time == None or month < chosen_treatment_time:
            time_treated = None
            treated = 0
        else:
            time_treated = chosen_treatment_time
            treated = 1

        evaluations = pd.concat([evaluations, pd.DataFrame({
            'patient_id': patient_id, 
            'pain': pain, 
            'urgency': urgency, 
            'frequency': frequency, 
            'time_since_entry': time_since_entry, 
            'time_treated': time_treated, 
            'treated': treated
        })])

evaluations.groupby('patient_id')[['pain', 'urgency', 'frequency']].mean().head()

,pain,urgency,frequency
patient_id,,,
0,4.6250,5.3750,8.8750
1,5.3750,4.5000,8.5000
2,4.5625,4.5625,7.3750
3,4.3125,5.1250,7.8125
4,3.5000,4.6250,8.3125


### Generate risk sets

In [358]:
risk_sets = {}

for time in evaluations['time_treated'].dropna().unique():
    treated = evaluations[(evaluations['time_since_entry'] == time) & (evaluations['time_treated'] == time)]
    untreated = evaluations[(evaluations['time_since_entry'] == time) & (evaluations['treated'] == 0)]

    risk_sets[time] = (treated, untreated)

risk_sets.keys()

dict_keys([33, 30, 24, 12, 48, 9, 27, 18, 3, 45, 21, 6, 36, 39, 42, 15])

### Add Binary Variables to risk set

In [359]:
def create_bin_var(df, variables):
    binary_vars =defaultdict(list)
    for var in variables:
        # Compute one-third and two-thirds percentiles
        lower_percentile = np.percentile(df[var], 33)
        upper_percentile = np.percentile(df[var], 67)

        # Create binary variables for lower, middle, and upper groups
        binary_lower = (df[var] <= lower_percentile).astype(int)
        binary_middle = ((df[var] > lower_percentile) & (df[var] <= upper_percentile)).astype(int)
        binary_upper = (df[var] > upper_percentile).astype(int)
        
        # Append to binary_vars
        binary_vars[f"{var}_lower"].extend(binary_lower)
        binary_vars[f"{var}_upper"].extend(binary_upper)

    # Generate binary DataFrame for each symptom variable
    binary_df = pd.DataFrame(binary_vars)
    return binary_df

### Update risk set 
Update both dataframes to join with their respective baseline features. Additionally, 12 binary variables are added to these dataframes to determine the feature quantile.

In [360]:
new_rs = {}
variables = [
    'pain_current',
    'urgency_current',
    'frequency_current',
    'pain_baseline',
    'urgency_baseline',
    'frequency_baseline'
]

for key, (t, u) in risk_sets.items():
    # Treated DataFrame
    treated = t.merge(baseline, on='patient_id', suffixes=['_current', '_baseline'])
    
    bin_t = create_bin_var(treated, variables)
    treated = pd.concat([treated, bin_t], axis=1)

    # Untreated DataFrame
    untreated = u.merge(baseline, on='patient_id', suffixes=['_current', '_baseline'])

    bin_u = create_bin_var(untreated, variables)
    untreated = pd.concat([untreated, bin_u], axis=1)

    new_rs[key] = (treated, untreated)

list(new_rs.values())[0][0].columns

Index(['patient_id', 'pain_current', 'urgency_current', 'frequency_current',
       'time_since_entry', 'time_treated', 'treated', 'gender',
       'pain_baseline', 'urgency_baseline', 'frequency_baseline',
       'pain_current_lower', 'pain_current_upper', 'urgency_current_lower',
       'urgency_current_upper', 'frequency_current_lower',
       'frequency_current_upper', 'pain_baseline_lower', 'pain_baseline_upper',
       'urgency_baseline_lower', 'urgency_baseline_upper',
       'frequency_baseline_lower', 'frequency_baseline_upper'],
      dtype='object')

### Compute for Covariate Distances using Mahalanobis

In [373]:
def compute_distance(treated, untreated, covariates):
    treated_matrix = treated[covariates].to_numpy()
    untreated_matrix = untreated[covariates].to_numpy()
    
    return cdist(treated_matrix, untreated_matrix, metric='mahalanobis')

In [374]:
distance_matrices = {}

for key, (t, u) in new_rs.items():
    covariates = [
        f"{var}_lower" for var in variables
    ] + [
        f"{var}_upper" for var in variables
    ]
    
    distance = compute_distance(t, u, covariates)
    distance_matrices[key] = distance
    print(distance)

distance_matrices.keys()

[[4.73117403 4.76920781 4.1747246  5.38154174 5.34150167 5.81183611
  5.80720815 4.60960345 4.44956564 4.17198617 5.64012198 3.72422226
  5.78363439 4.91226696 4.96756226 4.01741362 5.36309149 4.65037819
  5.51116185 5.1304113  5.97906289 5.33242862 5.09379161 5.41061831
  2.5119915  4.5073727  3.93926978 5.25914684 5.06049641 4.4596337
  5.50023603 4.01740301 5.25073334 5.75554082 4.72217941 5.40078192
  5.06877227 4.65291814 5.88297074 4.18836881 4.10755259 3.8542666
  4.83142458 4.08008557 5.2034756  4.85475286 5.61186508 5.79111349
  4.2852091  5.24497578 4.35226255 4.85475286 4.58313627 5.3161933
  4.23870157 4.01741362 5.83363442 5.40101395 5.45416295 4.2222589
  4.86683005 4.50127196 2.39721863 4.49934002 5.05625905 5.57678955
  4.97630348]
 [5.17171918 4.70783819 4.3862463  5.37148006 3.76252631 4.83927429
  5.515686   4.93467096 3.38712788 4.29257553 3.83004638 4.20684891
  5.45683032 4.91736172 5.6315748  4.81887492 5.22403542 5.35472847
  4.61093751 4.73817554 4.69599066 5.1

dict_keys([33, 30, 24, 12, 48, 9, 27, 18, 3, 45, 21, 6, 36, 39, 42, 15])

### Balance Risk Sets through Integer Programming Problem

In [363]:
def find_min_matches(distance_matrix):
    num_treated, num_untreated = distance_matrix.shape

    # Initialize the model
    model = Model("PatientMatching")

    # Define the decision variables (binary matrix)
    x = {}
    for i in range(num_treated):
        for j in range(num_untreated):
            # Create a binary variable for each treated-untreated pair (0 or 1)
            x[i, j] = model.addVar(vtype="B", name=f"match_{i}_{j}")

    # Objective: Minimize the total distance
    model.setObjective(
        sum(distance_matrix[i, j] * x[i, j] for i in range(num_treated) for j in range(num_untreated)),
        sense="minimize"
    )
    
    # Constraints:
    # 1. Each treated individual is matched to exactly one untreated individual
    for i in range(num_treated):
        model.addCons(sum(x[i, j] for j in range(num_untreated)) == 1)

    # 2. Each untreated individual is matched to at most one treated individual
    for j in range(num_untreated):
        model.addCons(sum(x[i, j] for i in range(num_treated)) <= 1)

    # Optimize the model using the SCIP solver
    model.optimize()
    # Extract the matched pairs if the solution is optimal
    if model.getStatus() == "optimal":
        matches = [(i, j) for i in range(num_treated) for j in range(num_untreated) if model.getVal(x[i, j]) > 0.5]
        return matches
    else:
        raise ValueError("No optimal solution found.")

In [370]:
matches = []

for matrix in distance_matrices.values():
    new_matches = find_min_matches(matrix)
    matches.extend(new_matches)

presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 78 upgd conss, 0 impls, 78 clqs
   (0.0s) probing: 51/737 (6.9%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) symmetry computation finished: 2 generators found (max: 1500, log10 of symmetry group size: 0.0) (symcode time: 0.00)
dynamic symmetry handling statistics:
   orbitopal reduction:       no components
   orbital reduction:         no components
   lexicographic reduction:    2 permutations with support sizes 22, 22
handled 2 out of 2 symmetry components
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 78 cliques
pres